In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(
    { "figure.figsize": (6, 4) },
    style='ticks',
    color_codes=True,
    font_scale=0.8
)
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings('ignore')

import sklearn
sklearn.set_config(transform_output="pandas")

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.preprocessing import OneHotEncoder, TargetEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#df = pd.read_parquet(
#    '/content/oct_2024.snappy.parquet'
#)
path = "/content/drive/MyDrive/AutoTaderHack/oct_2024.snappy.parquet"
df = pd.read_parquet(path)
df .head(5)

,stock_item_id,last_date_seen,first_date_seen,days_to_sell,first_retailer_asking_price,last_retailer_asking_price,can_home_deliver,reviews_per_100_advertised_stock_last_12_months,segment,seats,...,first_registration_date,attention_grabber,manufacturer_approved,price_indicator_rating,adjusted_retail_amount_gbp,predicted_mileage,number_of_images,first_image_label,advert_quality,postcode_area
0,33e3749b79a2847010ec1ad4716745393a2fd6b7d9db8d...,2024-10-07,2022-10-12,726,24495,18995,False,0.3,Independent,5.0,...,2016-09-09,None,False,GOOD,18716.0,73842.0,11,FRONT_RIGHT,72,BT
1,cbf6f80e01e939aa017a9641d02f5cc24f78c3e69d9a74...,2024-10-25,2023-05-13,531,16990,12990,False,0.9,Franchise,5.0,...,2021-01-14,£750 deposit contribution*,True,GOOD,13110.0,22740.0,12,FRONT_RIGHT,55,HA
2,c49040c0388b7533accbf7d4db144efbe7b2a251256b3b...,2024-10-05,2023-07-31,432,14481,11913,False,9.1,Independent,5.0,...,2016-12-17,SAT NAV • REAR CAMERA,False,GOOD,11873.0,55729.0,38,FRONT_RIGHT,41,BS
3,eacc77fecac6a43b328d0cbb16c192baa0654284a37b7d...,2024-10-21,2023-09-18,399,61000,36500,False,2.9,Franchise,5.0,...,2023-05-15,8.9 PERCENT APR AVAILABLE,False,GOOD,36809.0,10689.0,49,FRONT_RIGHT,71,PR
4,3618f09687bd940abafde2cbeab7181df5e1055d5c000d...,2024-10-25,2023-10-08,383,129995,109995,True,11.1,Independent,5.0,...,2019-03-24,*6 MONTHS FREE RAC WARRANTY*,False,NOANALYSIS,109023.0,34559.0,46,FRONT_LEFT,62,OL


#THE DATA


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155676 entries, 0 to 155675
Data columns (total 43 columns):
 #   Column                                           Non-Null Count   Dtype  
---  ------                                           --------------   -----  
 0   stock_item_id                                    155676 non-null  object 
 1   last_date_seen                                   155676 non-null  object 
 2   first_date_seen                                  155676 non-null  object 
 3   days_to_sell                                     155676 non-null  int32  
 4   first_retailer_asking_price                      155676 non-null  int64  
 5   last_retailer_asking_price                       155676 non-null  int64  
 6   can_home_deliver                                 155676 non-null  bool   
 7   reviews_per_100_advertised_stock_last_12_months  141413 non-null  float64
 8   segment                                          155577 non-null  object 
 9   seats          

In [ ]:
max_entries = 0
for col in df.columns:
    max_entries = max(max_entries, df[col].nunique())

print(f"The maximum number of entries across all features is: {max_entries}")


The maximum number of entries across all features is: 155676


In [ ]:
df_cleaned = df.copy()

In [ ]:
#drop unwanted columns after delibration.
'''df_cleaned = df_cleaned.drop(columns=[
    'stock_item_id','last_date_seen','first_date_seen','can_home_deliver', 'reviews_per_100_advertised_stock_last_12_months',
    'segment', 'seats', 'doors', 'co2_emission_gpkm', 'engine_power_bhp', 'length_mm',
    'boot_space_seats_up_litres', 'derivative_id', 'manufacturer_approved'
])'''


"df_cleaned = df_cleaned.drop(columns=[\n    'stock_item_id','last_date_seen','first_date_seen','can_home_deliver', 'reviews_per_100_advertised_stock_last_12_months',\n    'segment', 'seats', 'doors', 'co2_emission_gpkm', 'engine_power_bhp', 'length_mm',\n    'boot_space_seats_up_litres', 'derivative_id', 'manufacturer_approved'\n])"

#clean the data

In [ ]:
df_cleaned.isna().sum()

,0
stock_item_id,0
last_date_seen,0
first_date_seen,0
days_to_sell,0
first_retailer_asking_price,0
last_retailer_asking_price,0
can_home_deliver,0
reviews_per_100_advertised_stock_last_12_months,14263
segment,99
seats,87


##target= days_to_sell  

In [ ]:
#target
df_cleaned['days_to_sell'].describe()

,days_to_sell
count,155676.000000
mean,40.338781
std,52.537916
min,1.000000
25%,11.000000
50%,25.000000
75%,50.000000
max,1528.000000


In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df_cleaned['days_to_sell'], bins=30, kde=True)
plt.title('Frequency of Days to Sell')
plt.xlabel('Days to Sell')
plt.ylabel('Frequency')
plt.show()


In [ ]:
Q1 = data['days_to_sell'].quantile(0.1)
Q3 = data['days_to_sell'].quantile(0.9)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
data = data[(data['days_to_sell'] >= lower_bound) & (data['days_to_sell'] <= upper_bound)]
#this drops rows so do at end of cleaning


##AGE  both plate and first registartion date

regsistartion age missing 8
but vehile age is full.



In [ ]:
###### regsitration plate cleaned to create registration_age
import re
import datetime as dt

def has_letter(plate):
    return bool(re.search(r"[a-zA-Z]", str(plate)))

def letter_plate_to_age(plate):
    letter_to_year = {
        "A": 1983, "B": 1984, "C": 1985, "D": 1986, "E": 1987,
        "F": 1988, "G": 1989, "H": 1990, "J": 1991, "K": 1992,
        "L": 1993, "M": 1994, "N": 1995, "P": 1996, "R": 1997,
        "S": 1998, "V": 1999, "T": 1999, "W": 2000, "X": 2000,
        "Y": 2001
    }
    return dt.datetime.now().year - letter_to_year[plate]

def calculate_registration_age(plate_number, current_year):
    if 2000 + plate_number < current_year:
        return current_year - (2000 + plate_number)
    else:
        return current_year - (2000 + plate_number - 50)

current_year = dt.datetime.now().year

mask = df_cleaned["plate"].apply(has_letter)
df_cleaned_with_letter_plates = df_cleaned[mask]
df_cleaned_with_num_plates = df_cleaned[~mask]

df_cleaned_with_letter_plates["registration_age"] = df_cleaned_with_letter_plates["plate"].dropna().apply(letter_plate_to_age)
df_cleaned_with_letter_plates[["registration_age", "plate"]]

for index, row in df_cleaned_with_letter_plates.iterrows():
    df_cleaned.loc[index, "registration_age"] = row["registration_age"]

df_cleaned_with_num_plates["registration_age"] = df_cleaned_with_num_plates["plate"].astype(int).apply(lambda x: calculate_registration_age(x, current_year))

for index, row in df_cleaned_with_num_plates.iterrows():
    df_cleaned.loc[index, "registration_age"] = row["registration_age"]

In [ ]:
df_cleaned['first_registration_date'].isna().sum()

In [ ]:
########first_registration_date cleaned to create vehicle_age
df_cleaned['first_registration_date'] = pd.to_datetime(df_cleaned['first_registration_date'], errors='coerce')

current_year = pd.Timestamp.now().year

df_cleaned['vehicle_age'] = current_year - df_cleaned['first_registration_date'].dt.year
df_cleaned['vehicle_age'].isna().sum()
df_cleaned['vehicle_age'] = df_cleaned['vehicle_age'].fillna(df_cleaned['vehicle_age'].mean())

# Calculate Q1 and Q3 from the 'vehicle_age' column
Q1 = df_cleaned['vehicle_age'].quantile(0.25)
Q3 = df_cleaned['vehicle_age'].quantile(0.75)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [ ]:
df_cleaned['vehicle_age'].sample()

In [ ]:
sns.countplot(x='vehicle_age', data=df_cleaned)
plt.xticks(rotation=45, ha='right')  # If set_xticklabels is used above, this won't display anything


In [ ]:
df_cleaned['registration_age'].isna().sum()

In [ ]:
df_cleaned['vehicle_age'].isna().sum()

In [ ]:
print(df_cleaned[['vehicle_age', 'registration_age']])


In [ ]:
print(df_cleaned['vehicle_age'].describe())
print(df_cleaned['registration_age'].describe())


##transmission_type

In [ ]:
df_cleaned['transmission_type'].isna().sum()

In [ ]:
df_cleaned['transmission_type'].describe()

In [ ]:
df_cleaned['transmission_type'].value_counts()

In [ ]:
df_cleaned['transmission_type'] = df_cleaned['transmission_type'].replace('Unlisted', 'Automatic')

In [ ]:
########transmission_type
df_cleaned['transmission_type'] = df_cleaned['transmission_type'].map({'Automatic': 0, 'Manual': 1})
print(df_cleaned['transmission_type'].value_counts())
#transmission_type
#Automatic  78331
#Manual 77283

##drivetrain
i.e four wheel drive

In [ ]:
df_cleaned['drivetrain'].describe()

In [ ]:
df_cleaned['drivetrain'].isna().sum()

In [ ]:
df_cleaned['drivetrain'].value_counts()

In [ ]:
########drivetrain
df_cleaned['drivetrain'] = df_cleaned['drivetrain'].fillna(df_cleaned['drivetrain'].mode()[0])
drivetrain_mapping = {
    'Front Wheel Drive': 0,
    'Four Wheel Drive': 1,
    'Rear Wheel Drive': 2
}

df_cleaned['drivetrain'] = df_cleaned['drivetrain'].map(drivetrain_mapping)


##colour


In [ ]:
df_cleaned['colour'].isna().sum()


In [ ]:
df_cleaned['colour'].value_counts()

In [ ]:
# fills NAs by randomly assigning them
# based on the existing distribution of colours.
# This ensures that the dataset maintains its natural balance instead of overrepresenting GREY
colour_distribution = df_cleaned["colour"].value_counts(normalize=True)

# Sample missing values based on the existing distribution
missing_indices = df_cleaned[df_cleaned["colour"].isna()].index
df_cleaned.loc[missing_indices, "colour"] = np.random.choice(colour_distribution.index, size=len(missing_indices), p=colour_distribution.values)


In [ ]:
df_cleaned['colour'].isna().sum()

In [ ]:
df_cleaned['colour'].value_counts()

##FUEL Type- cleaned and label encoded


In [ ]:
df_cleaned['fuel_type'].isna().sum()

In [ ]:
df_cleaned['fuel_type'].describe()

In [ ]:
df_cleaned['fuel_type'].value_counts()

In [ ]:
# Replace 'Bi-Fuel' and 'Unlisted' with 'Petrol' in the 'fuel_type' column
df_cleaned['fuel_type'] = df_cleaned['fuel_type'].replace(['Bi-Fuel', 'Unlisted'], 'Petrol')


In [ ]:

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df_cleaned['fuel_type_encoded'] = le.fit_transform(df_cleaned['fuel_type'])
print(df_cleaned['fuel_type_encoded'].value_counts())
print(df_cleaned[['fuel_type', 'fuel_type_encoded']].head(20))


##odometer_reading_miles- IQR

In [ ]:
df_cleaned['odometer_reading_miles'].isna().sum()

In [ ]:
df_cleaned['odometer_reading_miles'].describe()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df_cleaned['odometer_reading_miles'], bins=30, kde=True)
plt.title('Distribution of Odometer Readings')
plt.xlabel('Odometer Reading (Miles)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
##odometer_reading_miles
#change from decimals to int
df_cleaned = df_cleaned[ # OR WE HIT THE IQR METHOD
    (df_cleaned['odometer_reading_miles'] >= (df_cleaned['odometer_reading_miles'].quantile(0.25) - 1.5 * (df_cleaned['odometer_reading_miles'].quantile(0.75) - df_cleaned['odometer_reading_miles'].quantile(0.25)))) &
    (df_cleaned['odometer_reading_miles'] <= (df_cleaned['odometer_reading_miles'].quantile(0.75) + 1.5 * (df_cleaned['odometer_reading_miles'].quantile(0.75) - df_cleaned['odometer_reading_miles'].quantile(0.25))))
]

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df_cleaned['odometer_reading_miles'], bins=30, kde=True)
plt.title('Distribution of Odometer Readings')
plt.xlabel('Odometer Reading (miles)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df_cleaned['odometer_reading_miles'].describe()

##predicted_mileage

In [ ]:
df_cleaned['predicted_mileage'].describe()

In [ ]:
df_cleaned['predicted_mileage'].isna().sum()

In [ ]:
df_cleaned['predicted_mileage'] = df_cleaned['predicted_mileage'].fillna(df_cleaned['odometer_reading_miles']+313)

##postcode_area

In [ ]:
df_cleaned['postcode_area'].describe()

In [ ]:
df_cleaned['postcode_area'].value_counts()

In [ ]:
df_cleaned['postcode_area'].isna().sum()

In [ ]:
df_cleaned['postcode_area'] = df_cleaned['postcode_area'].ffill()

In [ ]:
df_cleaned['postcode_area'].isna().sum()

##first_image_label - needs encoding

In [ ]:
df_cleaned['first_image_label'].isna().sum()

In [ ]:
df_cleaned['first_image_label'].describe()

In [ ]:
df_cleaned['first_image_label'] = df_cleaned['first_image_label'].fillna("FRONT_RIGHT")

In [ ]:
df_cleaned['first_image_label'].isna().sum()

In [ ]:
df_cleaned['first_image_label'].value_counts()

##first_retailer_asking_price + last_retailer_asking_price

In [ ]:
df_cleaned['first_retailer_asking_price'].isna().sum()

In [ ]:
df_cleaned['first_retailer_asking_price'].describe()


In [ ]:
df_cleaned['last_retailer_asking_price'].isna().sum()

In [ ]:
df_cleaned['last_retailer_asking_price'].describe()

In [ ]:

df_cleaned["first_retailer_asking_price"] = df_cleaned["first_retailer_asking_price"].astype(float)
df_cleaned["last_retailer_asking_price"] = df_cleaned["last_retailer_asking_price"].astype(float)
df_cleaned["asking_price_diff_rel"] = df_cleaned["last_retailer_asking_price"] / df_cleaned["first_retailer_asking_price"]

threshold_higher = 20
threshold_lower = 0.2

high_diff_mask = df_cleaned['asking_price_diff_rel'] > threshold_higher
low_diff_mask = df_cleaned["asking_price_diff_rel"] < threshold_lower

df_cleaned.loc[high_diff_mask, 'first_retailer_asking_price'] = df_cleaned.loc[high_diff_mask, 'last_retailer_asking_price']
df_cleaned.loc[low_diff_mask, 'last_retailer_asking_price'] = df_cleaned.loc[low_diff_mask, 'first_retailer_asking_price']


##adjusted_retail_amount_gbp

In [ ]:
df_cleaned['adjusted_retail_amount_gbp'].isna().sum()

In [ ]:
df_cleaned['adjusted_retail_amount_gbp'].describe()

In [ ]:
def fill_na_with_random(row):
    if pd.notna(row['adjusted_retail_amount_gbp']):
        return row['adjusted_retail_amount_gbp']
    elif row['first_retailer_asking_price'] < row['last_retailer_asking_price']:
        return np.random.randint(row['first_retailer_asking_price'], row['last_retailer_asking_price'])
    else:
        # Provide a default value if the range is not valid
        return (row['first_retailer_asking_price'] + row['last_retailer_asking_price']) / 2

df_cleaned['adjusted_retail_amount_gbp'] = df_cleaned.apply(fill_na_with_random, axis=1)

In [ ]:
df_cleaned['adjusted_retail_amount_gbp'].describe()

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
df_cleaned['adjusted_retail_amount_gbp'].describe()

##zero_to_sixty_mph_seconds

In [ ]:
df_cleaned['zero_to_sixty_mph_seconds'].isna().sum()

In [ ]:
df_cleaned['zero_to_sixty_mph_seconds'].describe()


In [ ]:
df_cleaned['zero_to_sixty_mph_seconds'] = pd.to_numeric(df_cleaned['zero_to_sixty_mph_seconds'], errors='coerce')
df_cleaned['zero_to_sixty_mph_seconds'] = df_cleaned['zero_to_sixty_mph_seconds'].astype('float64')


In [ ]:
mask = (df_cleaned["zero_to_sixty_mph_seconds"].isnull())
mode_top_speed = df_cleaned.groupby(['make', 'model'])['zero_to_sixty_mph_seconds'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

zero_top_speed_indices = df_cleaned[mask].index

for index in zero_top_speed_indices:
    make = df_cleaned.loc[index, 'make']
    model = df_cleaned.loc[index, 'model']

    mode_value = mode_top_speed[(make, model)]
    df_cleaned.loc[index, 'zero_to_sixty_mph_seconds'] = mode_value

##top_speed_mph

In [ ]:
df_cleaned['top_speed_mph'].isna().sum()

In [ ]:
df_cleaned['top_speed_mph'].describe()

In [ ]:
mode_top_speed = df_cleaned.groupby(['make', 'model'])['top_speed_mph'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

zero_top_speed_indices = df_cleaned[df_cleaned['top_speed_mph'] == 0].index

for index in zero_top_speed_indices:
    make = df_cleaned.loc[index, 'make']
    model = df_cleaned.loc[index, 'model']
    try:
        mode_value = mode_top_speed[(make, model)]
        df_cleaned.loc[index, 'top_speed_mph'] = mode_value
    except KeyError:
        print(f"Mode not found for group: {make}, {model}")

df_cleaned.loc[df_cleaned["top_speed_mph"] == 0, "top_speed_mph"] = np.nan



In [ ]:
df_cleaned['top_speed_mph'].isna().sum()

In [ ]:
df_cleaned['top_speed_mph'] = df_cleaned.groupby(['make', 'model'])['top_speed_mph'].transform(lambda x: x.fillna(x.mean()))


In [ ]:
df_cleaned['top_speed_mph'].isna().sum()

In [ ]:
df_cleaned['top_speed_mph'].fillna(df_cleaned['top_speed_mph'].mean(), inplace=True)


In [ ]:
df_cleaned['top_speed_mph'].isna().sum()

##fuel_economy_wltp_combined_mpg

In [ ]:
df_cleaned['fuel_economy_wltp_combined_mpg'].isna().sum()

In [ ]:
df_cleaned['fuel_economy_wltp_combined_mpg'].describe()

In [ ]:
mode_top_speed = df_cleaned.groupby(['make', 'model'])['fuel_economy_wltp_combined_mpg'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

mask = (df_cleaned["fuel_economy_wltp_combined_mpg"] == "0.0") | (df_cleaned["fuel_economy_wltp_combined_mpg"].isnull())
zero_top_speed_indices = df_cleaned[mask].index

for index in zero_top_speed_indices:
    make = df_cleaned.loc[index, 'make']
    model = df_cleaned.loc[index, 'model']
    try:
        mode_value = mode_top_speed[(make, model)]
        df_cleaned.loc[index, 'fuel_economy_wltp_combined_mpg'] = mode_value
    except Exception as e:
        print(f"Mode not found for group: {make}, {model}")


In [ ]:
df_cleaned['fuel_economy_wltp_combined_mpg'].isna().sum()

In [ ]:
df_cleaned['fuel_economy_wltp_combined_mpg'] = pd.to_numeric(df_cleaned['fuel_economy_wltp_combined_mpg'], errors='coerce')

df_cleaned['fuel_economy_wltp_combined_mpg'].fillna(df_cleaned['fuel_economy_wltp_combined_mpg'].mean(), inplace=True)

In [ ]:
df_cleaned['fuel_economy_wltp_combined_mpg'].isna().sum()

##attention_grabber

In [ ]:
df_cleaned['attention_grabber'].isna().sum()

In [ ]:
df_cleaned['attention_grabber'].describe()

In [ ]:
df_cleaned['attention_grabber_category'] = np.nan

df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('service', case=False, na=False), 'attention_grabber_category'] = 'service'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('cockpit', case=False, na=False), 'attention_grabber_category'] = 'cockpit'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('delivery', case=False, na=False), 'attention_grabber_category'] = 'delivery'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('exchange', case=False, na=False), 'attention_grabber_category'] = 'exchange'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('part-ex', case=False, na=False), 'attention_grabber_category'] = 'exchange'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('heated seat', case=False, na=False), 'attention_grabber_category'] = 'heated seats'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('finance', case=False, na=False), 'attention_grabber_category'] = 'finance'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('reserve', case=False, na=False), 'attention_grabber_category'] = 'reserve'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('mileage', case=False, na=False), 'attention_grabber_category'] = 'mileage'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('warranty', case=False, na=False), 'attention_grabber_category'] = 'warranty'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('apr', case=False, na=False), 'attention_grabber_category'] = 'apr'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('sat nav', case=False, na=False), 'attention_grabber_category'] = 'satnav'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('satnav', case=False, na=False), 'attention_grabber_category'] = 'satnav'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('Satellite Navigation', case=False, na=False), 'attention_grabber_category'] = 'satnav'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('bluetooth', case=False, na=False), 'attention_grabber_category'] = 'bluetooth'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('android', case=False, na=False), 'attention_grabber_category'] = 'android'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('sale', case=False, na=False), 'attention_grabber_category'] = 'sale'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('today', case=False, na=False), 'attention_grabber_category'] = 'today'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('styl', case=False, na=False), 'attention_grabber_category'] = 'styling'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('carplay', case=False, na=False), 'attention_grabber_category'] = 'carplay'
df_cleaned.loc[df_cleaned['attention_grabber'].str.contains('cruise', case=False, na=False), 'attention_grabber_category'] = 'cruise control'


In [ ]:
df_cleaned.loc[df_cleaned['attention_grabber_category'].isna(), 'attention_grabber_category'] = 'other'

In [ ]:
mask = df_cleaned["insurance_group"].isnull()
mode_insurance = df_cleaned.groupby(['make', 'model'])['insurance_group'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

insurance_indices = df_cleaned[mask].index

for index in insurance_indices:
    make = df_cleaned.loc[index, 'make']
    model = df_cleaned.loc[index, 'model']

    try:
        mode_value = mode_insurance[(make, model)]
        df_cleaned.loc[index, 'insurance_group'] = mode_value
    except Exception as e:
        df_cleaned.loc[index, 'insurance_group'] = np.nan

In [ ]:
Q1 = df_cleaned['days_to_sell'].quantile(0.1)
Q3 = df_cleaned['days_to_sell'].quantile(0.9)

# Calculate IQR
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame
df_cleaned = df_cleaned[(df_cleaned['days_to_sell'] >= lower_bound) & (df_cleaned['days_to_sell'] <= upper_bound)]

# Visualize the filtered data
sns.histplot(data=df_cleaned, x="days_to_sell")
plt.title('Days To Sell (Following IQR)')

#cleaned most

In [ ]:
df_cleaned.isna().sum()

In [ ]:
df_cleaned.info()

In [ ]:
#sns.lineplot(x=np.mean('first_retailer_asking_price'), y='days_to_sell', data=df_cleaned)

In [ ]:
#sns.lineplot(x='last_retailer_asking_price', y='days_to_sell', data=df_cleaned)

In [ ]:
sns.boxplot(x='seats', y='days_to_sell', data=df_cleaned, hue='seats')

In [ ]:
sns.barplot(x='doors', y='days_to_sell', data=df_cleaned)

In [ ]:
sns.barplot(x='first_image_label', y='days_to_sell', data=df_cleaned, hue='first_image_label')
plt.xticks(rotation=60, ha='right')


In [ ]:
sns.barplot(x='first_image_label', y='days_to_sell', data=df_cleaned, hue='first_image_label')

# Remove x-axis labels
plt.gca().set_xticklabels([])  # This clears the x-axis labels

# Rotate ticks (optional, only if you want to ensure rotation won't affect anything)
plt.xticks(rotation=60, ha='right')  # If set_xticklabels is used above, this won't display anything

# Show the plot with legend
categories = df_cleaned['first_image_label'].unique()
plt.legend(labels=categories,loc='upper left', bbox_to_anchor=(1,1));  # Ensure the legend conveys the category information
plt.show()

left to do

1.   drop the ones we dont need no more at the end
2.   clean up insurance_group, generation



In [ ]:
mask = df_cleaned["fuel_type"].isin(["Electric", "Diesel Plug-in Hybrid", "Petrol Plug-in Hybrid"])
electric = df_cleaned[mask].drop(columns=["attention_grabber"]).dropna()
data = df_cleaned[~mask].drop(columns=["battery_range_miles", "battery_usable_capacity_kwh", "attention_grabber"]).dropna()

In [ ]:
hybrid = electric[electric["fuel_type"].isin(["Petrol Plug-in Hybrid", "Diesel Plug-in Hybrid"])]
electric = electric[~electric["fuel_type"].isin(["Petrol Plug-in Hybrid", "Diesel Plug-in Hybrid"])]

In [ ]:
data.info()

# Fi

# Electric Model

In [ ]:
electric.info()

In [ ]:
electric['price_percentile'] = pd.qcut(electric['first_retailer_asking_price'], 10, labels=False)
electric = electric[pd.to_numeric(electric['price_percentile'], errors='coerce').notnull()]
electric['price_percentile'] = electric['price_percentile'].astype(int)

In [ ]:
electric_categorical = ['can_home_deliver',
 'insurance_group',
 'body_type',
 'fuel_type',
 'transmission_type',
 'drivetrain',
 'colour',
 'manufacturer_approved',
 'price_indicator_rating',
 'first_image_label',
 'postcode_area',
 'attention_grabber_category']

In [ ]:
def group_diff(df, group_cols, value_col):
    group_means = df.groupby(group_cols)[value_col].mean()
    df = df.merge(group_means.rename(value_col + '_group_mean'), left_on=group_cols, right_index=True)
    df['diff_' + value_col] = df[value_col] / df[value_col + '_group_mean']
    return df

for col in electric.select_dtypes(include=np.number).columns:
    electric = group_diff(electric, ['price_percentile'], col)

electric

In [ ]:
def mmg_avgs(dataset, cols):
    for col in cols:
        if col in dataset.columns and col != "days_to_sell":
            groups = dataset.groupby(["make", "model", "generation", col])
            dataset["mmg_" + col + "_dts_avg"] = groups["days_to_sell"].transform("mean")
            dataset = dataset.drop(columns=[col])

    return dataset.drop(columns=['days_to_sell', 'make', 'model', 'generation'])

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error

from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.ensemble import VotingRegressor

ridge = Ridge()
lasso = Lasso()
linear = LinearRegression()
forest = RandomForestRegressor()
knn = KNeighborsRegressor()
gradient = GradientBoostingRegressor()

ensemble = VotingRegressor(estimators=[('ridge', ridge), ('lasso', lasso),('forest', forest), ('knn',knn)])

X = electric[[
 'insurance_group',
 'body_type',
 'fuel_type',
 'transmission_type',
 'drivetrain',
 'colour',
 'price_indicator_rating',
 'first_image_label',
 'postcode_area',
 'attention_grabber_category',
  'diff_battery_range_miles',
 'asking_price_diff_rel',
 'diff_top_speed_mph',
  "days_to_sell",
  "make",
  "model",
 "generation"]]
y = electric[["days_to_sell"]]

scaler = StandardScaler()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_test = mmg_avgs(X_train, electric_categorical), mmg_avgs(X_test,electric_categorical)

ensemble.fit(X_train, y_train)

y_pred = ensemble.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)

print(f"Ensemble MSE: {mse}")
print(f"Ensemble R2: {r2}")
print(f"Ensemble MAE: {mae}")
print(f"Ensemble MedAE: {medae}")

In [ ]:
# prompt: test best combination of models for voting regressor

from sklearn.ensemble import VotingRegressor, GradientBoostingRegressor

ridge = Ridge()
lasso = Lasso()
linear = LinearRegression()
forest = RandomForestRegressor()
knn = KNeighborsRegressor()
gradient = GradientBoostingRegressor()

# Define the models to test in the VotingRegressor
estimators = [
    ('ridge', ridge),
    ('lasso', lasso),
    ('linear', linear),
    ('forest', forest),
    ('knn', knn),
    ('gradient', gradient),
]

ensemble_results = pd.DataFrame()

best_r2 = -1
best_ensemble = None

# Iterate through combinations of models (excluding single models)
for i in range(1, 1 << len(estimators)):  # Iterate through all possible subsets
    current_estimators = []
    for j in range(len(estimators)):
      if (i >> j) & 1:
        current_estimators.append(estimators[j])

    if len(current_estimators) > 1: # exclude single model combinations
        ensemble = VotingRegressor(estimators=current_estimators)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        X_train, X_test = mmg_avgs(X_train, electric_categorical), mmg_avgs(X_test,electric_categorical)

        ensemble.fit(X_train, y_train.values.ravel())
        y_pred = ensemble.predict(X_test)

        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        medae = median_absolute_error(y_test, y_pred)
        cross_val_test = cross_val_score(ensemble, X_test, y_test, cv=5)

        print(f"Current Estimators: {[name for name, _ in current_estimators]}")
        print(f"Ensemble MSE: {mse}")
        print(f"Ensemble R2: {r2}")
        print(f"Ensemble MAE: {mae}")
        print(f"Ensemble MedAE: {medae}")
        print(f"Cross Val Score: {cross_val_test}")


        if r2 > best_r2:
            best_r2 = r2
            best_ensemble = ensemble

print(f"\nBest Ensemble R2: {best_r2}")
#print(f"Best Ensemble: {best_ensemble}") #You can access the best model if you need it later



In [ ]:
hybrid['price_percentile'] = pd.qcut(hybrid['first_retailer_asking_price'], 10, labels=False)
hybrid = hybrid[pd.to_numeric(hybrid['price_percentile'], errors='coerce').notnull()]
hybrid['price_percentile'] = hybrid['price_percentile'].astype(int)

In [ ]:
for col in hybrid.select_dtypes(include=np.number).columns:
    hybrid = group_diff(hybrid, ['price_percentile'], col)

hybrid

In [ ]:
X = hybrid[[
 'insurance_group',
 'colour',
 'price_indicator_rating',
 'postcode_area',
 'attention_grabber_category',
 'asking_price_diff_rel_group_mean',
 'diff_asking_price_diff_rel',
  "days_to_sell",
  "make",
  "model",
 "generation"]]
y = hybrid[["days_to_sell"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_test = mmg_avgs(X_train, electric_categorical), mmg_avgs(X_test,electric_categorical)

ensemble = VotingRegressor(estimators=[('ridge', ridge), ('lasso', lasso), ('linear', linear), ('forest', forest), ('knn',knn)])
ensemble.fit(X_train, y_train)

y_pred = ensemble.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)

print(f"Ensemble MSE: {mse}")
print(f"Ensemble R2: {r2}")
print(f"Ensemble MAE: {mae}")
print(f"Ensemble MedAE: {medae}")

In [ ]:
from sklearn.ensemble import VotingRegressor, GradientBoostingRegressor

# ... (Your existing code)

ridge = Ridge()
lasso = Lasso()
linear = LinearRegression()
forest = RandomForestRegressor()
knn = KNeighborsRegressor()
gradient = GradientBoostingRegressor()

# Define the models to test in the VotingRegressor
estimators = [
    ('ridge', ridge),
    ('lasso', lasso),
    ('linear', linear),
    ('forest', forest),
    ('knn', knn),
    ('gradient', gradient),
]

ensemble_results = pd.DataFrame()

best_r2 = -1
best_ensemble = None

# Iterate through combinations of models (excluding single models)
for i in range(1, 1 << len(estimators)):  # Iterate through all possible subsets
    current_estimators = []
    for j in range(len(estimators)):
      if (i >> j) & 1:
        current_estimators.append(estimators[j])

    if len(current_estimators) > 1: # exclude single model combinations
        ensemble = VotingRegressor(estimators=current_estimators)

        # ... (Your existing code to fit, predict, and evaluate)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        X_train, X_test = mmg_avgs(X_train, electric_categorical), mmg_avgs(X_test,electric_categorical)

        ensemble.fit(X_train, y_train.values.ravel()) #add ravel to fix data conversion warning
        y_pred = ensemble.predict(X_test)

        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        medae = median_absolute_error(y_test, y_pred)
        cross_val_test = cross_val_score(ensemble, X_test, y_test, cv=5)

        print(f"Current Estimators: {[name for name, _ in current_estimators]}") #Print estimators used in current iteration
        print(f"Ensemble MSE: {mse}")
        print(f"Ensemble R2: {r2}")
        print(f"Ensemble MAE: {mae}")
        print(f"Ensemble MedAE: {medae}")
        print(f"Cross Val Score: {cross_val_test}")


        if r2 > best_r2:
            best_r2 = r2
            best_ensemble = ensemble

print(f"\nBest Ensemble R2: {best_r2}")
#print(f"Best Ensemble: {best_ensemble}") #You can access the best model if you need it later



In [ ]:
results = X_test.copy()
results["days_to_sell_predicted"] = y_pred
results["days_to_sell"] = y_test

In [ ]:
results.columns

In [ ]:
sns.scatterplot(data=results, x="days_to_sell", y="days_to_sell_predicted")

In [ ]:
results["rel_pred_diff"] = results["days_to_sell_predicted"] / results["days_to_sell"]

In [ ]:
# prompt: get most inaccurate rows from hybrid df. get original rows assocaited with results from hybrid df using index

# Assuming 'results' DataFrame from the provided code is available

# Get the most inaccurate rows (largest relative prediction difference)
most_inaccurate_rows = results.sort_values("rel_pred_diff", ascending=False).head(10)  # Adjust 10 to desired number

# Get the original row indices from the hybrid DataFrame
original_indices = most_inaccurate_rows.index

# Access the original rows in the 'hybrid' DataFrame using the indices
original_rows = hybrid.loc[original_indices]

# Print or further process the original_rows
original_rows


In [ ]:

asking_price_diffs = hybrid.loc[X_test.index, 'diff_asking_price_diff_rel']

results["diff_asking_price_diff_rel"] = asking_price_diffs


In [ ]:
asking_price_diffs = hybrid.loc[X_test.index, 'diff_asking_price_diff_rel']
results["diff_asking_price_diff_rel"] = asking_price_diffs

# Calculate quantiles and IQR
Q1 = results['diff_asking_price_diff_rel'].quantile(0.25)
Q3 = results['diff_asking_price_diff_rel'].quantile(0.75)
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers
filtered_results = results[
    (results['diff_asking_price_diff_rel'] >= lower_bound) &
    (results['diff_asking_price_diff_rel'] <= upper_bound)
]

# Create scatter plot with filtered data
sns.scatterplot(data=filtered_results, x="days_to_sell", y="days_to_sell_predicted", hue="asking_price_diff_rel_group_mean")

In [ ]:
sns.scatterplot(data=results, x="days_to_sell", y="days_to_sell_predicted", hue="diff_asking_price_diff_rel")

# Main Model

In [ ]:
data.info()

In [ ]:
def mmg_avgs(dataset, col):
  if col != "days_to_sell": #check if column is not days_to_sell before grouping
    groups = dataset.groupby(["make", "model", "generation", col])
    dataset["mmg_" + col +"_dts_avg"] = groups["days_to_sell"].transform("mean")
  else:
  # If col is "days_to_sell", create a dummy column with the same values
    dataset["mmg_" + col + "_dts_avg"] = dataset["days_to_sell"]

  return dataset[["mmg_" + col +"_dts_avg"]]

In [ ]:
categorical = ['can_home_deliver',
 'insurance_group',
 'body_type',
 'fuel_type',
 'transmission_type',
 'drivetrain',
 'colour',
 'manufacturer_approved',
 'price_indicator_rating',
 'first_image_label',
 'postcode_area',
 'attention_grabber_category',
  "body_type",
  "fuel_type",
  "transmission_type"]

In [ ]:
grouped_data_list = pd.DataFrame()

for col in categorical:
  sample = data[["days_to_sell", "make", "model", "generation", col]].copy()
  X = sample[["days_to_sell", "make", "model", "generation",col]]
  y = sample[["days_to_sell"]]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  X_train, X_test = mmg_avgs(X_train, col), mmg_avgs(X_test, col)
  model = RandomForestRegressor()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  grouped_data_list =pd.concat([grouped_data_list, pd.DataFrame({"feature": col, "mse": mean_squared_error(y_test, y_pred), "r2": r2_score(y_test, y_pred)}, index=[0])])

In [ ]:
grouped_data_list.sort_values("r2")

In [ ]:
from sklearn.preprocessing import StandardScaler

numeric_features = data.select_dtypes(include=np.number).columns
results = pd.DataFrame()
scaler = StandardScaler()
for feature in numeric_features:
  sample = electric[[feature, "days_to_sell"]].dropna()
  X = scaler.fit_transform(sample[[feature]])
  y = sample[["days_to_sell"]]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  model = RandomForestRegressor()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  results =pd.concat([results, pd.DataFrame({"feature": feature, "mse": mean_squared_error(y_test, y_pred), "r2": r2_score(y_test, y_pred)}, index=[0])])

results.sort_values(by="r2", ascending=False)

In [ ]:
def main_mmg_avgs(dataset, cols):
    for col in cols:
        if col in dataset.columns and col != "days_to_sell":
            groups = dataset.groupby(["make", "model", "generation", col])
            dataset["mmg_" + col + "_dts_avg"] = groups["days_to_sell"].transform("mean")
            dataset = dataset.drop(columns=[col]) #Commented out to keep original columns

    return dataset.drop(columns=["make","model","generation","days_to_sell"])


In [ ]:
main_categorical = ["postcode_area", "attention_grabber_category", "price_indicator_rating", "colour", "insurance_group", "first_image_label"]

In [ ]:
X = data[[
 "postcode_area",
 "attention_grabber_category",
 "price_indicator_rating",
 "colour",
 "insurance_group",
 "first_image_label",
 'asking_price_diff_rel',
  "days_to_sell",
  "make",
  "model",
 "generation"]].sample(frac=0.25)
y = X[["days_to_sell"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_test = main_mmg_avgs(X_train, main_categorical), main_mmg_avgs(X_test,main_categorical)

ridge = Ridge()
lasso = Lasso()
linear = LinearRegression()
forest = RandomForestRegressor()
knn = KNeighborsRegressor()
gradient = GradientBoostingRegressor()

estimators = [
    ('ridge', ridge),
    ('lasso', lasso),
    ('linear', linear),
    ('forest', forest),
    ('knn', knn),
    ('gradient', gradient),
]

ensemble_results = pd.DataFrame()

best_cross_val = -1
best_ensemble = None

for i in range(1, 1 << len(estimators)):
    current_estimators = []
    for j in range(len(estimators)):
      if (i >> j) & 1:
        current_estimators.append(estimators[j])

    if len(current_estimators) > 1:
        ensemble = VotingRegressor(estimators=current_estimators)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        X_train, X_test = mmg_avgs(X_train, electric_categorical), mmg_avgs(X_test,electric_categorical)

        ensemble.fit(X_train, y_train.values.ravel())
        y_pred = ensemble.predict(X_test)

        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        medae = median_absolute_error(y_test, y_pred)
        cross_val_test = cross_val_score(ensemble, X_test, y_test, cv=5)

        print(f"Current Estimators: {[name for name, _ in current_estimators]}")
        print(f"Ensemble MSE: {mse}")
        print(f"Ensemble R2: {r2}")
        print(f"Ensemble MAE: {mae}")
        print(f"Ensemble MedAE: {medae}")
        print(f"Cross Val Score: {cross_val_test.mean()}")


        if cross_val_test.mean() > best_cross_val:
            best_cross_val = cross_val_test.mean()
            best_ensemble = ensemble

print(f"\nBest Ensemble R2: {best_cross_val}")
print(f"Best Ensemble: {best_ensemble}")

In [ ]:
X = data[[
 "postcode_area",
 "attention_grabber_category",
 "price_indicator_rating",
 "colour",
 "insurance_group",
 "first_image_label",
 'asking_price_diff_rel',
  "days_to_sell",
  "make",
  "model",
 "generation"]]
y = data[["days_to_sell"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_test = main_mmg_avgs(X_train, main_categorical), main_mmg_avgs(X_test,main_categorical)

best_ensemble.fit(X_train, y_train)

y_pred = best_ensemble.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)

print(f"Ensemble MSE: {mse}")
print(f"Ensemble R2: {r2}")
print(f"Ensemble MAE: {mae}")
print(f"Ensemble MedAE: {medae}")